# Fixed-bed problem


Jialu Wang (jwang44@nd.edu) and Alex Dowling (adowling@nd.edu)

University of Notre Dame

This notebook conducts design of experiments for a fixed-bed model with the Pyomo.DOE.
    

In [1]:
from pyomo.environ import *
from pyomo.dae import *

import idaes

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from itertools import permutations, product, combinations

### Import Pyomo.DOE

In [2]:
from fim_doe import *

### Import fixed-bed model

In [3]:
from fixed_bed_model import *

alpha option:
1 / (1 + exp(-x))
    Import ComponentSet from pyomo.common.collections.  (deprecated in 5.7.1)
    (called from <frozen importlib._bootstrap>:219)


### Set up fixed-bed model 

In [4]:
def add_model(doe, timepoints, time_start=0):
    '''
    add variables, equations and discretize the model time
    Arguments
    ---------
    timepoints: the timesteps
    time_start: where the timesteps start. For self defined timepoints it's 0. For experimental data it's -10.
    '''
    add_variables(doe, timesteps=timepoints, start=time_start)
    add_equations(doe)
    print ('the number of timepoints is', NFEt)
    
def discretize(doe, no_points=68, collo=False):
    '''
    Discretization 
    Arguments 
    ---------
    no_points: how many time invertals to divide. For self defined timepoints it's 69. For experimental data it's 68. 
    here it's 68
    collo: if using collocation or finite difference
    '''
    if collo:
        TransformationFactory('dae.collocation').apply_to(doe, nfe=no_points, ncp=3, scheme='LAGRANGE-RADAU', wrt=doe.t)
    else:
        TransformationFactory('dae.finite_difference').apply_to(doe, nfe=no_points, scheme='BACKWARD', wrt=doe.t)
        
    for i in doe.t: 
        doe.Q[i].fix()
    
    
def initialize(doe, init_point):
    '''
    Initialize the fixed bed 
    Arguments 
    ---------
    init_point: initial point, csv file
    '''
    print('The init point is', init_point)    
    store = pd.read_csv(init_point)
    position_max = store['position'].max()
    store['position'] = store['position'] / position_max

    initial_bed_csv(doe, store)
    fix_initial_bed(doe)
    
def generate_exp_addQ(feed, y, const_q):
    '''
    Generalize experiments adding Q
    Arguments 
    ---------
    feed: feed temperature, [K]
    yfeed: feed CO2 fraction, mol % 
    Q: extra heat added
    '''
    initialize_q = {}
    for t in exp_time_:
        initialize_q[t] = const_q
    dv_dict_overall = {'temp_feed': {0: feed}, 'yfeed': {0: y}, 'Q': initialize_q}
    return dv_dict_overall

def generate_exp(feed, y):
    '''
    Generalize experiments with no adding Q
    Arguments 
    ---------
    feed: feed temperature, [K]
    yfeed: feed CO2 fraction, mol % 
    '''
    dv_dict_overall = {'temp_feed': {0: feed}, 'yfeed': {0: y}}
    return dv_dict_overall

In [5]:
NFEt = 68 # No. of time points 
time_scale = 3200 # [s], time
collo = False # if not using collocation, finite difference is used 
exp_time_ = time_points(NFEt, time_scale) # The discretized time points 

scena_all = {'scena-name': [0], 'fitted_transport_coefficient':212, 'ua': np.log(5.0E6)}

# Starting point for the fixed-bed model 
initialize_point = '/media/psf/Home/fixed_bed_adsorption/fixed-bed-saved-results/2021oct8_f293_b293_y15_5e6.csv'


In [6]:
def model_integrate(scena, args=[True, False], exp_time=exp_time_, init_point=initialize_point):
    '''Work as the model creation function provided to Pyomo.DOE
    Note: When more than one Pyomo function are needed for creating, initializing, and discretizing
    models, they should be combined in this way and provide a ready-for-solving model name
    Arguments
    ---------
    args_list: Additional arguments for fixed-bed 
    exp_time: discretized time points 
    init_point: starting point for solving the model 
    
    Return 
    ------
    test: model for Pyomo.DOE
    '''
    test = create_model(scena, temp_feed=313.15, temp_bath=293.15, y=0.15, doe_model=args[0], k_aug=args[1], opt=False, diff=0)
    add_model(test, exp_time)
    discretize(test)
    initialize(test, init_point)
    return test


### Specify inputs for Pyomo.DOE

In [7]:
# specify parameters 
parameter_dict = {'fitted_transport_coefficient':212, 'ua': np.log(5.0E6)}

# specify design variables 
#dv_pass = {'temp_feed': None, 'yfeed': None, 'Q':exp_time_}
dv_pass = {'temp_feed':None, 'yfeed': None}
#if_optimization = {'temp_feed': True, 'yfeed': True, 'Q':False}

# specify measurements 
t_measure = exp_time_
measurement_pass = {'FCO2': {19: t_measure}, 'temp': {10: t_measure, 19:t_measure}}
var = {'FCO2': {19: 0.00001}, 'temp': {10: 1, 19:1}}

measure_class = Measurements(measurement_pass, variance=var)

All measurements are flattened.
Flatten measurement name: ['FCO2_index_19', 'temp_index_10', 'temp_index_19']


In [8]:
# empty prior
prior_all = np.zeros((2,2))

prior_pass=np.asarray(prior_all)

#L_initials = np.linalg.cholesky(prior_pass)
#print(L_initials)

print('The prior information FIM:', prior_pass)
print('Prior Det:', np.linalg.det(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigvals(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigh(prior_pass)[1])

The prior information FIM: [[0. 0.]
 [0. 0.]]
Prior Det: 0.0
Eigenvalue of the prior experiments FIM: [0. 0.]
Eigenvalue of the prior experiments FIM: [[1. 0.]
 [0. 1.]]


## Compute FIM
Compute a square MBDoE problem for a specified experiment

In [9]:
# choose from 'sequential_finite', 'direct_kaug'
sensi_opt = 'sequential_finite'
#sensi_opt = 'direct_kaug'

# specify model for different modes
if (sensi_opt == 'sequential_finite'):
    args_ = [True, False]
    
elif (sensi_opt =='direct_kaug'):
    args_ = [True, True]
    
# define an experiment
#exp1 = generate_exp(293.15, 0.15, 0)
exp1 = generate_exp(293.15, 0.15)

In [10]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measure_class, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_)

# Read the solution stored
read_name = '/media/psf/Home/fixed_bed_adsorption/fixed-bed-saved-results/20220205_293_293_40_5e6' 

# use .compute_FIM() function
result = doe_object.compute_FIM(exp1, mode=sensi_opt, FIM_store_name = 'fixed_bed.csv', 
                                store_output = None, read_output=read_name,
                                extract_single_model = extract3_v2,
                                formula='central')

# analyze the result object
result.calculate_FIM(doe_object.design_values)
    
    

<class 'numpy.ndarray'>
[[ 0.00000000e+00  1.28471638e-08  1.12713652e-08  2.63982484e-08
   1.02040044e-07  3.98517310e-07  1.51958731e-06  6.10916240e-06
   2.49894675e-05  7.15157383e-05  1.58050236e-04  3.16991824e-04
   3.03062803e-04  2.56325129e-04  2.00260909e-04  1.45172187e-04
   9.55752596e-05  5.31279856e-05  1.80647232e-05 -1.00675560e-05
  -3.20226220e-05 -4.86505240e-05 -6.07911433e-05 -6.92215983e-05
  -7.46335121e-05 -7.76267419e-05 -7.87119432e-05 -7.83175786e-05
  -7.67988460e-05 -7.44470901e-05 -7.14989093e-05 -6.81445514e-05
  -6.45354211e-05 -6.07906535e-05 -5.70027813e-05 -5.32425640e-05
  -4.95630651e-05 -4.60030665e-05 -4.25899096e-05 -3.93418451e-05
  -3.62699652e-05 -3.33797848e-05 -3.06725276e-05 -2.81461685e-05
  -2.57962715e-05 -2.36166605e-05 -2.15999528e-05 -1.97379798e-05
  -1.80221151e-05 -1.64435296e-05 -1.49933848e-05 -1.36629790e-05
  -1.24438542e-05 -1.13278720e-05 -1.03072655e-05 -9.37467227e-06
  -8.52315196e-06 -7.74619294e-06 -7.03771016e-06 -6

In [11]:
print('======Result summary======')
print('Four design criteria log10() value:')
print('A-optimality:', np.log10(result.trace))
print('D-optimality:', np.log10(result.det))
print('E-optimality:', np.log10(result.min_eig))
print('Modified E-optimality:', np.log10(result.cond))

======Result summary======
Four design criteria log10() value:
A-optimality: -1.24348554660971
D-optimality: -3.2823204172307037
E-optimality: -1.9417477192947035
Modified E-optimality: 0.6011750213587033


In [12]:
# Addiontional: to store the solution for one specific model 
#dataframe = extract3_v2(doe_object.models[0])
#dataframe.to_csv('test6.csv')

# Additional: to show the breakthrough curve for adsorption 
#breakthrough_modify2(doe_object.models[0])

## Gradient-based optimization 

In [13]:
# define a starting point for optimization 
exp1 = generate_exp(313.15, 0.15)

### Sequential initial solution 

In [14]:
fim_init = result.FIM
print(fim_init)
    
l_init = np.linalg.cholesky(fim_init)
print(l_init)

[[ 0.04493459 -0.00489063]
 [-0.00489063  0.01214942]]
[[ 0.2119778   0.        ]
 [-0.02307143  0.10778279]]


In [ ]:
args_ = [True, False]

doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measure_class, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_)

square_result, optimize_result = doe_object.optimize_doe(exp1, if_optimize=True, if_Cholesky=True,
                                                         scale_nominal_param_value=True, objective_option='det',
                                                         L_initial=l_init, fim_initial=fim_init)


Sensitivity information is scaled by its corresponding parameter nominal value.
The inlet feed density is 38.40726649935368 [mol/m3]
the number of timepoints is 68
The init point is /media/psf/Home/fixed_bed_adsorption/fixed-bed-saved-results/2021oct8_f293_b293_y15_5e6.csv
Model # of time grid is 69 , initial point # of time grid is 69
    'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes
max_iter=3000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipop

  96r 1.8959336e+00 2.36e+00 2.82e+02   0.6 1.53e-02   4.3 1.00e+00 1.00e+00f  1
  97r 1.8937870e+00 2.35e+00 8.82e+03   0.6 8.17e-02   3.8 1.00e+00 1.00e+00f  1
  98r 1.8929833e+00 2.34e+00 7.12e+03   0.6 5.05e-02   4.2 1.00e+00 1.00e+00f  1
  99r 1.8917021e+00 2.33e+00 2.88e+03   0.6 1.47e-01   3.7 1.00e+00 5.32e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 100r 1.8907999e+00 2.32e+00 3.08e+03   0.6 6.30e-02   4.2 1.00e+00 1.00e+00f  1
 101r 1.8888847e+00 2.31e+00 1.07e+04   0.6 1.89e-01   3.7 1.00e+00 7.09e-01f  1
 102r 1.8878226e+00 2.30e+00 8.96e+03   0.6 4.19e-01   3.2 1.00e+00 1.32e-01f  1
 103r 1.8856045e+00 2.28e+00 5.49e+03   0.6 3.59e-02   3.6 1.00e+00 7.33e-01f  1
 104r 1.8765923e+00 2.19e+00 2.54e+04   0.6 1.81e-01   3.2 1.00e+00 1.00e+00f  1
 105r 1.8761766e+00 2.19e+00 3.54e+03   0.6 4.08e-02   4.5 1.00e+00 1.00e+00f  1
 106r 1.8749148e+00 2.17e+00 1.45e+04   0.6 3.19e-02   4.0 1.00e+00 1.00e+00f  1
 107r 1.8744419e+00 2.17e+00

 233r-1.4175077e-01 4.61e-04 1.43e-02  -3.3 3.22e-04    -  1.00e+00 1.00e+00h  1
 234r-6.1216232e-01 5.27e-02 1.10e+02  -4.9 7.68e-01    -  4.60e-01 7.13e-01f  1
 235r-1.9055243e+00 3.61e-01 1.39e+02  -4.9 4.54e+00  -2.0 3.56e-01 5.27e-01f  1
 236r-2.8175124e+00 5.10e-01 2.84e+02  -4.9 7.91e+00    -  7.56e-02 3.66e-01f  1
 237r-3.5257016e+00 5.63e-01 3.77e+02  -4.9 7.01e+00    -  1.91e-01 5.11e-01f  1
 238r-3.9568341e+00 3.71e-01 3.76e+02  -4.9 4.49e+00    -  3.74e-01 7.32e-01f  1
 239r-4.0683801e+00 9.64e-02 4.74e+02  -4.9 1.58e+00    -  2.61e-01 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 240r-4.0564027e+00 4.11e-02 3.17e+02  -4.9 1.06e-01    -  3.87e-01 8.23e-01h  1
 241r-4.0437297e+00 1.28e-02 2.49e+02  -4.9 3.91e-02    -  3.42e-01 1.00e+00f  1
 242r-4.0406125e+00 8.52e-03 1.42e+02  -4.9 2.56e-02    -  4.41e-01 1.00e+00h  1
 243r-4.0399794e+00 6.45e-04 3.54e+01  -4.9 1.88e-03    -  7.76e-01 1.00e+00h  1
 244r-4.0393837e+00 3.94e-04

    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
<class 'numpy.ndarray'>
[[ 4.07808128e-29 -3.94763269e-03  3.42293190e-03  1.98647228e-06
  -4.63458094e-03  4.76616321e-03  3.27473697e-04  1.46826284e-06
  -5.19881497e-03  5.68470435e-03  1.00528039e-05 -3.95140781e-03
   1.24808245e-04  9.68443365e-04  5.00408428e-07 -5.37958720e-03
   7.19626140e-03  4.09306587e-05 -3.26401938e-03 -2.57369427e-03
   2.14042564e-03  6.50187972e-07 -5.02915349e-03  6.20646161e-03
   1.49632536e-04  2.88407008e-06 -4.51691529e-03  4.22240612e-03
   3.53800369e-06 -4.41346211e-03  2.98975649e-03  4.87998602e-04
   1.02285628e-06 -5.35709350e-03  7.49174749e-03  1.60595732e-05
  -3.71861480e-03 -9.70998948e-04  1.24423648e-03  4.11028458e-07
  -5.30834718e-03  5.53700255e-03  6.50726318e-05 -3.04617797e-03
  -3.20085408e-03  2.86850553e-03  1.14563342e-06 -4.8420529

  35 -1.0758612e+01 7.18e+00 5.02e+06  -1.0 6.14e-03   5.4 1.49e-01 6.15e-01h  1
  36 -1.0758598e+01 4.96e+00 1.25e+07  -1.0 1.21e-02   4.9 9.95e-01 3.10e-01h  1
  37 -1.0758594e+01 4.31e+00 3.34e+06  -1.0 8.15e-02   4.5 1.00e+00 1.32e-01h  1
  38 -1.0758592e+01 4.10e+00 3.77e+06  -1.0 1.93e-01   4.0 1.00e+00 4.94e-02h  1
  39 -1.0758589e+01 3.57e+00 3.16e+06  -1.0 7.67e-02   3.5 4.63e-01 1.29e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  40 -1.0758586e+01 3.06e+00 2.31e+06  -1.0 3.98e-02   4.8 1.00e+00 1.44e-01h  1
  41 -1.0758586e+01 3.05e+00 2.29e+06  -1.0 4.60e+00   4.4 9.73e-02 1.20e-04h  5
  42 -1.0758578e+01 1.94e+00 1.44e+06  -1.0 1.95e-02   4.8 1.00e+00 3.65e-01h  1
  43 -1.0758566e+01 7.55e-03 4.27e+06  -1.0 1.09e-03   7.0 6.91e-03 1.00e+00f  1
Reallocating memory for MA57: lfact (27900470)
Reallocating memory for MA57: lfact (31608857)
  44 -1.0758566e+01 5.90e-03 1.49e+07  -1.0 1.02e-02   6.5 2.93e-02 1.00e+00h  1
  45 -1.0729564

 128r-1.2167918e+01 2.88e+02 1.30e+08  -0.1 1.47e-01   8.0 1.43e-01 9.39e-02h  1
 129r-1.2167918e+01 2.45e+02 2.02e+08  -0.1 1.97e-01   7.5 1.15e-02 1.52e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 130r-1.2167918e+01 2.43e+02 2.11e+08  -0.1 7.43e-01   7.9 3.90e-02 9.69e-03h  1
 131r-1.2167918e+01 2.20e+02 3.20e+08  -0.1 1.18e+00   7.4 3.53e-02 9.61e-02h  1
 132r-1.2167918e+01 2.16e+02 3.16e+08  -0.1 1.36e+00   7.9 4.62e-02 1.92e-02h  1
 133r-1.2167918e+01 1.87e+02 2.80e+08  -0.1 1.40e+00   7.4 1.78e-01 1.33e-01h  1
 134r-1.2167918e+01 1.87e+02 2.80e+08  -0.1 6.22e+00   6.9 6.21e-03 1.47e-03h  1
 135r-1.2167918e+01 1.39e+02 2.14e+08  -0.1 1.23e+00   7.3 1.13e-02 2.64e-01h  1
 136r-1.2167918e+01 1.31e+02 2.02e+08  -0.1 9.11e-01   6.9 1.45e-01 5.63e-02h  1
 137r-1.2167918e+01 1.30e+02 2.01e+08  -0.1 8.59e-01   7.3 5.87e-01 6.64e-03h  1
 138r-1.2167918e+01 1.16e+02 1.82e+08  -0.1 8.52e-01   7.7 1.54e-02 1.04e-01h  1
 139r-1.2167918e+01 9.13e+01

 266r-1.2167918e+01 1.80e+00 1.82e+04  -0.1 3.85e-01   3.5 8.85e-01 1.10e-01f  1
 267r-1.2167918e+01 1.77e+00 1.35e+04  -0.1 1.25e-01   4.0 1.00e+00 2.56e-01f  1
 268r-1.2167918e+01 1.70e+00 9.93e+03  -0.1 3.47e-01   3.5 1.00e+00 2.15e-01f  1
 269r-1.2167918e+01 1.67e+00 6.34e+03  -0.1 1.30e-01   3.9 1.00e+00 3.30e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 270r-1.2167918e+01 1.60e+00 4.36e+03  -0.1 3.73e-01   3.4 1.00e+00 2.01e-01f  1
 271r-1.2167918e+01 1.56e+00 2.33e+03  -0.1 1.43e-01   3.9 1.00e+00 3.70e-01f  1
 272r-1.2167918e+01 1.50e+00 1.88e+03  -0.1 4.12e-01   3.4 1.00e+00 1.44e-01f  1
 273r-1.2167918e+01 1.44e+00 1.01e+03  -0.1 1.54e-01   3.8 1.00e+00 4.32e-01f  1
 274r-1.2167918e+01 1.30e+00 1.00e+03  -0.1 4.63e-01   3.3 1.00e+00 2.88e-01f  1
 275r-1.2167918e+01 1.17e+00 1.37e+03  -0.1 1.39e+00   2.9 1.00e+00 8.81e-02f  1
 276r-1.2167918e+01 1.03e+00 9.99e+02  -0.1 5.21e-01   3.3 1.00e+00 2.68e-01f  1
 277r-1.2167918e+01 8.71e-01

In [ ]:
print('======Result summary======')
print('This optimization is solved with status:', optimize_result.status)
print('It gives solution:', optimize_result.solution)
print('The log10(OF) optimized is:', optimize_result.obj_value)
print('The result FIM is:', optimize_result.FIM)

t_list = []
for t in optimize_result.model.t:
    t_list.append(t)

T_list = []
for i in t_list:
    T_list.append(value(optimize_result.model.T[i]))

si=16
plt.rc('axes', titlesize=si)
plt.rc('axes', labelsize=si)
plt.rc('xtick', labelsize=si)
plt.rc('ytick', labelsize=si)
plt.rc('legend', fontsize=12)
    
plt.plot(t_list, T_list, 'b', linewidth=2)
#plt.scatter(t_list, T_list, 'b')
plt.ylabel('T [$K$]')
plt.xlabel('Time [$h$]')
plt.show()

## Exploratory analysis 

### Define enumeration ranges

In [ ]:
# Define design ranges 
dv_apply_name = ['temp_feed','yfeed']
# go over these designs
design_ranges = [list(np.linspace(293,353,13)), list(np.linspace(0.1,0.4,13))]
# control time points 
dv_apply_time = [[0],[0]]

# Define experiments
exp1 = generate_exp(313.15, 293.15, 0.15, 0)
    
# choose from 'simultaneous', 'sequential', 'sipopt'
sensi_opt = 'sequential_finite'
#sensi_opt = 'direct_kaug'

### Enumeration

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measure_class, createmod,
                                prior_FIM=prior_pass, discretize_model=None, args=args_)

all_fim = doe_object.run_grid_search(exp1, design_ranges,dv_apply_name, dv_apply_time, 
                                     mode=sensi_opt, tee_option=True,
                                    scale_nominal_param_value=True, 
                                     store_name='20211031_heatmap'
                                    )

### Draw heatmaps

In [ ]:
fixed = {}
all_fim.figure_drawing(fixed, ['temp_feed','y'], 'fixed bed', '$T_{feed}$ [K]', 'y [mol%]' )